# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Spring 2025** </center>
---
### <center> **Streamer Structrure for final Proyect** </center>
### <center> **Par de Foraneos** </center>


In [19]:
import findspark
findspark.init()

2025-05-12 18:55:00+00:00,CAT,327.0181382463461,327.12688353502125,326.77868946266864,327.0154349547865
2025-05-12 18:55:00+00:00,NVDA,116.68253086339759,116.70881873872364,116.63233479695164,116.69098356208696
2025-05-12 18:55:00+00:00,AAPL,198.55665562119083,198.56633182310873,198.48974441806396,198.55080984815442
2025-05-12 18:55:00+00:00,CVX,138.54968709660383,138.6018135402931,138.51607539598896,138.51814390536353


## Spark Session creation


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Kafka") \
    .master("spark://a5ab6bdab4b3:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
sc = spark.sparkContext

2025-05-12 19:00:00+00:00,CAT,327.0354854086611,327.64527853652817,327.0354854086611,327.56878857797597
2025-05-12 19:00:00+00:00,NVDA,116.6873079926524,116.74495456659108,116.64531461390361,116.6977597401094
2025-05-12 19:00:00+00:00,AAPL,198.53962184763785,198.5644734186918,198.4833925707221,198.50417313060967
2025-05-12 19:00:00+00:00,CVX,138.53235671394935,138.57211982431014,138.5144232078006,138.54076313311487
2025-05-12 19:05:00+00:00,CAT,327.59701239496377,327.7456515762768,327.4392232625602,327.6503196953646
2025-05-12 19:05:00+00:00,NVDA,116.69573663998152,116.70974675343011,116.59480900847127,116.60374856328208
2025-05-12 19:05:00+00:00,AAPL,198.5144988482145,198.53155298564033,198.43754926173443,198.44181215986592
2025-05-12 19:05:00+00:00,CVX,138.54448003258375,138.58804551940486,138.50838842657498,138.52575437152282
2025-05-12 19:10:00+00:00,CAT,327.7791958834199,328.07592568370507,327.6033102133447,327.89082368006063
2025-05-12 19:10:00+00:00,NVDA,116.59669705463192,116.6

## Kafka Stream creation

In [ ]:
streamer_lines = []

for i in range(4):
    streamer_lines.append( spark \
                            .readStream \
                            .format("kafka") \
                            .option("kafka.bootstrap.servers", "69b1b3611d90:9093") \
                            .option("subscribe", f"stock_topic{i}") \
                            .load()
    )


2025-05-12 19:20:00+00:00,CAT,328.3383565225041,328.4761284417207,328.110511428169,328.2565975456787
2025-05-12 19:20:00+00:00,NVDA,116.53057387820438,116.6107903312397,116.49581246769795,116.54039420514415
2025-05-12 19:20:00+00:00,AAPL,198.52275365518227,198.55817771545372,198.4864289979123,198.52177934802708
2025-05-12 19:20:00+00:00,CVX,138.36900820944138,138.38880893584934,138.15681564227148,138.17075922803335
2025-05-12 19:25:00+00:00,CAT,328.302692322649,328.5261719256734,327.7350556295704,328.3335539519778
2025-05-12 19:25:00+00:00,NVDA,116.53145750250054,116.63446064807768,116.50484132131666,116.62678791161103
2025-05-12 19:25:00+00:00,AAPL,198.52714900995412,198.61722141833346,198.52034215743635,198.56644403243692
2025-05-12 19:25:00+00:00,CVX,138.18502238034233,138.20016865677167,138.128732568233,138.1570987590148


## Transform binary data into string

In [ ]:
from pyspark.sql.functions import col, split
from pyspark.sql.types import DoubleType, TimestampType

streamer_df = []

for i in range(4):
    df = streamer_lines[i].withColumn("value_str", col("value").cast("string"))
    df = df.withColumn("split", split(col("value_str"), ","))

    df = df.withColumn("timestamp", col("split").getItem(0).cast(TimestampType())) \
           .withColumn("company", col("split").getItem(1)) \
           .withColumn("open", col("split").getItem(2).cast(DoubleType())) \
           .withColumn("high", col("split").getItem(3).cast(DoubleType())) \
           .withColumn("low", col("split").getItem(4).cast(DoubleType())) \
           .withColumn("close", col("split").getItem(5).cast(DoubleType())) \
           .select("timestamp", "company", "open", "high", "low", "close")

    streamer_df.append(df)


2025-05-12 19:30:00+00:00,CAT,328.40335037084066,328.8992839281827,328.35120946522295,328.8242549806638
2025-05-12 19:30:00+00:00,NVDA,116.63584748752409,116.76947711899578,116.62215561925015,116.69429598813947
2025-05-12 19:30:00+00:00,AAPL,198.5626241863513,198.62632639017653,198.53522208600992,198.58760407999048
2025-05-12 19:30:00+00:00,CVX,138.16212680257655,138.18548319973985,138.09969785960598,138.1221749034623
2025-05-12 19:35:00+00:00,CAT,328.90046485203,329.31428673064465,328.8927510176115,329.1142473943186
2025-05-12 19:35:00+00:00,NVDA,116.68136289325305,116.70236661325787,116.60350854092155,116.6059388682124
2025-05-12 19:35:00+00:00,AAPL,198.58807240425142,198.64454151315704,198.55162764357334,198.58164995558437
2025-05-12 19:35:00+00:00,CVX,138.10774865053972,138.12868280549247,138.04837087650515,138.0958224886256


### Sink configuration

In [ ]:
query = []

for i in range(4):
    query.append(
        streamer_df[i] \
        .writeStream \
        .outputMode("append") \
        .trigger(processingTime='8 seconds') \
        .format("parquet") \
        .option("path", f"/home/jovyan/notebooks/data/final_project_ParDeForaneos/output{i}/")
        .option("checkpointLocation", f"/home/jovyan/notebooks/data/final_project_ParDeForaneos/checkpoints/stock_topic{i}") \
        .start()
    )

    #query[i].awaitTermination(100)


25/05/12 18:40:10 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/12 18:40:10 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/05/12 18:40:10 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/12 18:40:10 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/05/12 18:40:10 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/12 18:40:10 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied

25/05/12 18:40:11 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


2025-05-12 18:50:00+00:00,CAT,327.6509840539299,327.80042994149255,327.427456114019,327.67577390910975
2025-05-12 18:50:00+00:00,NVDA,116.12905209862667,116.16117288468372,116.04440355428993,116.05870076966745
2025-05-12 18:50:00+00:00,AAPL,199.54043554473478,199.5664565163892,199.39507385923227,199.49692455628022
2025-05-12 18:50:00+00:00,CVX,139.83020542838827,139.86122083174016,139.76412036864588,139.85516062739052
2025-05-12 18:55:00+00:00,CAT,327.6537409747842,328.0005825182821,327.54932376185496,327.9111742506819
2025-05-12 18:55:00+00:00,NVDA,116.07809383525657,116.16249653415947,116.043011205068,116.11853596862163
2025-05-12 18:55:00+00:00,AAPL,199.50426447548722,199.7323328192364,199.50162340250992,199.68857832641356
2025-05-12 18:55:00+00:00,CVX,139.86221637343903,140.12284084465688,139.8546140511195,140.10358589964696
2025-05-12 19:00:00+00:00,CAT,327.96164546360086,328.2128269975589,327.8293925347731,328.18605387760556
2025-05-12 19:00:00+00:00,NVDA,116.12281096800125,116.1

2025-05-12 19:45:00+00:00,CAT,327.9326115154557,328.30744701739263,327.745786409132,327.745786409132
2025-05-12 19:45:00+00:00,NVDA,115.89754295939105,115.96518211409271,115.84483814858184,115.94419367196501
2025-05-12 19:45:00+00:00,AAPL,200.02549030686532,200.0862859591267,199.95444218788657,200.00148021704285
2025-05-12 19:45:00+00:00,CVX,139.76597874543356,139.84194390581965,139.73067248702583,139.81562599682394
2025-05-12 19:50:00+00:00,CAT,327.7335627032576,328.44676898465235,327.59917864917793,328.3514478968355
2025-05-12 19:50:00+00:00,NVDA,115.95338553456777,115.97986158447029,115.73634482423532,115.85136201579672
2025-05-12 19:50:00+00:00,AAPL,200.00909041817982,200.00909041817982,199.75535640224686,199.76138529137856
2025-05-12 19:50:00+00:00,CVX,139.79314860570926,139.79314860570926,139.64751310465968,139.77489102015832
2025-05-12 19:55:00+00:00,CAT,328.39219160728624,328.98254294357827,328.38164972187593,328.98254294357827
2025-05-12 19:55:00+00:00,NVDA,115.85918890674671,

2025-05-12 20:05:00+00:00,CAT,329.3774582696384,329.6784419202879,329.26010378759213,329.31840670653605
2025-05-12 20:05:00+00:00,NVDA,116.09564219152833,116.09684920406075,115.87780085337195,115.93309485104666
2025-05-12 20:05:00+00:00,AAPL,199.63698680919225,199.703855416926,199.62140252677156,199.62862704099018
2025-05-12 20:05:00+00:00,CVX,139.57633880191563,139.68300729881835,139.50486314378034,139.68300729881835
2025-05-12 20:10:00+00:00,CAT,329.2597280483809,329.9403254351688,329.2351447727418,329.77150563035906
2025-05-12 20:10:00+00:00,NVDA,115.9181232690183,115.95083061858793,115.86169946572458,115.92474259869145
2025-05-12 20:10:00+00:00,AAPL,199.61786904934715,199.9078223618624,199.59570358351297,199.84880789501364
2025-05-12 20:10:00+00:00,CVX,139.69895040418785,139.86278237135812,139.61821590540228,139.86192518514883
2025-05-12 20:15:00+00:00,CAT,329.7875191479673,330.26938071685424,329.71701627877275,330.130220253311
2025-05-12 20:15:00+00:00,NVDA,115.92627453074148,116.

In [ ]:
for i in range(4):
    query[i].stop()

2025-05-12 20:20:00+00:00,CAT,330.1160574792095,330.1247303390892,330.1160574792095,330.1247303390892
2025-05-12 20:20:00+00:00,NVDA,116.18874239346606,116.18874239346606,116.18284668389056,116.18284668389056
2025-05-12 20:20:00+00:00,AAPL,199.95690682101622,199.9666952474855,199.95690682101622,199.9666952474855
2025-05-12 20:20:00+00:00,CVX,139.87175135349463,139.88233055160575,139.87175135349463,139.88233055160575
2025-05-12 18:40:00+00:00,CAT,330.1247303390892,330.2977135493049,329.98389936681184,330.2977135493049
2025-05-12 18:40:00+00:00,NVDA,116.18284668389056,116.19139588096895,116.16989340655095,116.16989340655095
2025-05-12 18:40:00+00:00,AAPL,199.9666952474855,200.02934393570837,199.94034501700787,200.00968270366118
2025-05-12 18:40:00+00:00,CVX,139.88233055160575,139.9708824599246,139.82215204745518,139.89266373001212
2025-05-12 18:45:00+00:00,CAT,330.2810340315601,330.5946673663702,330.2810340315601,330.5244729816865
2025-05-12 18:45:00+00:00,NVDA,116.17056186968705,116.171

In [32]:
sc.stop()